# Function: Trim to Relevant Part - Case Type Specific

In [0]:
import re


def trim_judgment(full_text):
    """
    Sequentially applies text trimming based on the presence of specific keywords or their absence.

    Search for "书记员" with the constraint that the characters can be spaced out but must all occur within 20 characters of the sequence.
    Search for "查明" and, if found, trim everything before this keyword in the already trimmed text (from the "书记员" search).
    If "查明" is not found, trim the first third of the text that might have been already trimmed based on the "书记员" search.

    Returns:
    - trimmed_text (str): The result of the trimming operation.
    - trimmedType (str): The method of trimming ('书记员', '查明', or 'last 2/3').
    - text_around_trimmed_point (str): Context around the point of trimming.
    """

    trim_a_third_switch = True

    if full_text is None or not isinstance(full_text, str):
        return None, "ERROR", ""

    # Initialize default values
    trimmed_text = full_text
    text_around_trimmed_point = ""
    trimmedType = "None"

    # Search for "书记员" with characters within 20 characters of each other
    matches = list(re.finditer(r"书\s{0,18}记\s{0,18}员", full_text))
    if matches:
        
        # Take the last match since we are interested in the occurrence starting from the end
        clerk_index = matches[-1].end()
        # Context around "书记员"
        start_pos = max(0, clerk_index - 10)
        end_pos = min(len(trimmed_text), clerk_index + 10)
        text_around_trimmed_point = trimmed_text[start_pos:end_pos]
        
        trimmed_text = trimmed_text[:clerk_index]  # Trim everything after the last "书记员"
        trimmedType = "书记员"


    # Search for "查明" in the already trimmed text
    find_index = trimmed_text.find("查明")
    if find_index != -1:
        # Context around "查明"
        start_pos = max(0, find_index - 10)
        end_pos = min(len(trimmed_text), find_index + 10)
        text_around_trimmed_point = trimmed_text[start_pos:end_pos]
        
        trimmed_text = trimmed_text[find_index:]  # Trim before "查明"
        trimmedType = "查明"

    elif trimmedType != "查明":  # Only trim 1/3 if "查明" wasn't found
        if trim_a_third_switch:
            one_third_length = len(trimmed_text) // 3
            
            # Context around the trimmed point
            start_pos = max(0, one_third_length - 10)
            end_pos = min(len(trimmed_text), one_third_length + 10)
            text_around_trimmed_point = trimmed_text[start_pos:end_pos]
            
            # Trim the first 1/3
            trimmed_text = trimmed_text[one_third_length:]
            trimmedType = "last 2/3"


    return trimmed_text, trimmedType, text_around_trimmed_point

# Function: Post Request and fetch Claude Response

In [0]:
import time
import requests
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType

api_key = dbutils.secrets.get(scope = "OhMyGPTAPI", key = "OhMyGPTAPI1")

def trim_and_fetch_facts(judgment_text):
    
    trimmed_judgment_text, trimmedType, text_around_trimmed_point = trim_judgment(judgment_text)

    url = "https://cfcus02.opapi.win/v1/messages"
    payload = json.dumps({
        "model": "claude-3-haiku-20240307",
        "stream": False,
        "system": "user will give you a legal judgement in Chinese. Please extract the name and amount of drugs from the judgment and output it within a JSON object. Use JSON format with the keys drug_a, amount_a, and drug_b, amount_b. Be concise Do not add anything else to the reply. If you can't extract the kind of drug or amount. reply 'NA'. reply using Chinese.",
        "messages": [
            {"role": "user", "content": trimmed_judgment_text}, 
            {"role": "assistant", "content": "{"}
            ],
        "max_tokens": 4096
    })
    headers = {
        "Authorization": 'Bearer ' + api_key,
        'Content-Type': 'application/json'
    }

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=20)
            response.raise_for_status()
            parsed_response = json.loads(response.text)
            # Check if the expected keys/path exists
            if "content" in parsed_response and len(parsed_response["content"]) > 0 and "text" in parsed_response["content"][0]:
                response_text = parsed_response["content"][0]["text"]
            else:
                raise KeyError("Unexpected response structure")
            return response_text, trimmedType, text_around_trimmed_point
        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"Error on attempt {attempt+1}: {e}")
        time.sleep(2 ** attempt + 1)

    # Default values in case of failure
    return "Failed to fetch response from API", "Error", "Error"



In [0]:
#API test block:
text_0 = "浙江省嵊州市人民法院 刑 事 判 决 书 （2019）浙0683刑初659号 公诉机关浙江省嵊州市人民检察院。 被告人王超，男，1991年11月17日出生，汉族，高中文化，无业，住宁夏回族自治区中卫市沙坡头区。因吸毒于2018年6月6日被嵊州市公安局行政拘留十一日，因吸毒于2018年11月21日被宁波市公安局奉化分局行政拘留十五日，因吸毒于2019年8月19日被嵊州市公安局行政拘留八日、强制戒毒二年。因本案于2019年8月10日被嵊州市公安局刑事拘留，9月16日被逮捕。现羁押于嵊州市看守所。 嵊州市人民检察院以嵊检二部刑诉[2019]56号起诉书指控被告人王超犯贩卖毒品罪，于2019年11月22日向本院提起公诉，并适用认罪认罚从宽制度，建议对被告人王超判处有期徒刑六个月，并处罚金人民币二千元。本院于11月25日立案，依法适用简易程序，实行独任审判，公开开庭审理了本案。嵊州市人民检察院指派检察员陈园出庭支持公诉，被告人王超到庭参加诉讼。现已审理终结。 嵊州市人民检察院指控，2018年7月20日下午，姚某用妻子单某的微信与被告人王超联系，委托被告人王超购买毒品甲基苯丙胺（冰毒），并支付被告人王超人民币650元。被告人王超从他人处获得0.1克甲基苯丙胺,于21日下午放置在嵊州市三江城&ldquo;东方巨龙&rdquo;对面的旗杆下，并将放置位置告诉姚某，由姚某捡走吸食。 2019年8月9日，被告人王超自动到嵊州市公安局甘霖派出所投案，如实供述了自己的犯罪事实。 上述事实，被告人王超在开庭审理过程中亦无异议，并有证人姚某、单某、王某、尹某的证言，微信转账记录，辨认笔录及照片，归案经过，情况说明，户籍证明，嵊州市公安局行政处罚决定书、强制隔离戒毒决定书等证据证实，足以认定。 本院认为，被告人王超违反毒品管理法律法规，贩卖毒品，其行为已构成贩卖毒品罪，应处三年以下有期徒刑，并处罚金。鉴于其犯罪以后自动投案，如实供述自己的罪行，是自首，予以从轻处罚。公诉机关指控的罪名成立，提出的量刑意见基本适当。根据其犯罪的事实、犯罪的性质、情节、对于社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一、四款、第三百五十七条、第六十七条第一款、第六十四条之规定，判决如下： 一、被告人王超犯贩卖毒品罪，判处有期徒刑六个月，并处罚金人民币二千元（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自二0一九年八月十日起至二0二0年二月九日止；罚金限判决生效后十日内缴纳）。 二、追缴被告人王超犯罪所得人民币650元，上缴国库。 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向浙江省绍兴市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。 审判员　　单学忠 二〇一九年十二月三日 书记员　　马生燕 附：《中华人民共和国刑法》相关法条： 第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。 走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产： （一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的； （二）走私、贩卖、运输、制造毒品集团的首要分子； （三）武装掩护走私、贩卖、运输、制造毒品的； （四）以暴力抗拒检查、拘留、逮捕，情节严重的； （五）参与有组织的国际贩毒活动的。 走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。 走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。 单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。 利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。 对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。 第三百五十七条本法所称的毒品，是指鸦片、海洛因、甲基苯丙胺（冰毒）、吗啡、大麻、可卡因以及国家规定管制的其他能够使人形成瘾癖的麻醉药品和精神药品。 毒品的数量以查证属实的走私、贩卖、运输、制造、非法持有毒品的数量计算，不以纯度折算。 第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。 被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。 犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。 第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 更多数据：www.macrodatas.cn"

text_1 ="云南省石屏县人民法院刑 事 判 决 书（2019）云2525刑初147号公诉机关石屏县人民检察院。被告人马威茂（绰号“小马”），男，1983年8月3日生，回族，初中文化，云南省峨山县人，客车驾驶员，住云南省峨山县。因涉嫌犯贩卖毒品罪，于2019年6月1日被刑事拘留，同年7月5日被逮捕，现羁押于石屏县看守所。辩护人张荣华，云南旭晖律师事务所律师。石屏县人民检察院以石检一部刑诉[2019]60号起诉书指控被告人马威茂犯贩卖毒品罪，于2019年10月21日向本院提起公诉。本院立案受理后，依法组成合议庭，于2019年12月19日公开开庭进行了审理。石屏县人民检察院委派查林娇、李涵出庭执行职务，被告人马威茂及其辩护人张荣华到庭参加诉讼。本案现已审理终结。石屏县人民检察院指控：被告人马威茂违反国家对毒品管理的规定，向多人多次贩卖甲基苯丙胺共计11.94克，其行为触犯了《中华人民共和国刑法》第三百四十七条第一款、第三款、第七款的规定，应当以贩卖毒品罪追究其刑事责任。被告人马威茂被抓获后如实供述其犯罪事实，系坦白，可以从轻处罚。建议对被告人马威茂在有期徒刑九年至十一年之间量刑，并处罚金30000元至50000元。公诉机关就指控的事实提交了相应的证据进行质证。被告人马威茂对公诉机关的指控无异议，当庭自愿认罪，但辩解其感觉没有贩卖过这么多毒品，请求从轻判处。辩护人张荣华对公诉机关指控被告马威茂犯贩卖毒品罪无异议。但辩护认为：马威茂本人系吸毒人员，量刑时应考虑其吸食毒品的情节；鉴于本案系零星交易毒品的案件，由于人的记忆力的局限性，不排除重复计算的可能性，难以精准确认毒品的数量，在量刑时应按照有利于被告人的原则对其从轻处罚；被告人马威茂一贯表现良好、系初犯，具有坦白情节。建议对马威茂在有期徒刑三年至七年之间量刑，并处罚金。经审理查明：被告人马威茂本人系吸毒人员，从事客车驾驶员工作。2018年至2019年5月30日期间，被告人马威茂向多人多次贩卖甲基苯丙胺片剂，具体事实如下：一、贩卖给李某（一）2019年4、5月份的一天，被告人马威茂在石屏县文化广场其客车上，以160元的价格贩卖甲基苯丙胺片剂4粒（计0.36克）给李某。（二）2019年4、5月份的一天，被告人马威茂在石屏县文化广场其客车上，以200元的价格贩卖甲基苯丙胺片剂6粒（计0.54克）给李某。（三）2019年4、5月份的一天，被告人马威茂在石屏县上，以200元的价格贩卖甲基苯丙胺片剂6粒（计0.54克）给李某。（四）2019年4、5月份，被告人马威茂在石屏县文化广场其客车上，先后2次以100元3粒的价格贩卖甲基苯丙胺片剂共计6粒（计0.54克）给李某。（五）2019年5月28日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。（六）2019年5月29日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。（七）2019年5月30日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。二、贩卖给管某（一）2019年4月份，被告人马威茂先后3次以100元3粒的价格贩卖甲基苯丙胺片剂共计9粒（计0.81克）给管某。（二）2019年5月下旬的一天，被告人马威茂在石屏县文化广场其客车上，以450元的价格贩卖甲基苯丙胺片剂15粒（计1.35克）给管某。（三）2019年5月27日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给管某。（四）2019年5月29日14时许，被告人马威茂在石屏县文化广场其客车上，以300元的价格贩卖甲基苯丙胺片剂10粒（计0.9克）给管某。三、贩卖给董某1（一）2019年5月19日13时30分许，被告人马威茂在石屏县文化广场其客车上，以120元的价格贩卖甲基苯丙胺片剂4粒（计0.36克）给董某1。（二）2019年5月22日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某1。（三）2019年5月22日13时30分许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某1。（四）2019年5月22日23时许，被告人马威茂在峨山县，以600元的价格贩卖甲基苯丙胺片剂18粒（计1.62克）给董某1。四、贩卖给董某2（一）2019年5月28日14时许，被告人马威茂在石屏县文化广场其客车上，以200元的价格贩卖甲基苯丙胺片剂5粒（计0.45克）给董某2。（二）2019年5月29日14时许，被告人马威茂在石屏县文化广场其客车上，以50元的价格贩卖甲基苯丙胺片剂2粒（计0.18克）给董某2。五、贩卖给解某12018年以来，被告人马威茂先后2次贩卖甲基苯丙胺片剂共计6粒（计0.54克）给解某1。六、贩卖给解某22019年5月下旬的一天，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给解某2。七、贩卖给董某32019年5月29日14时许，被告人马威茂在石屏县仁和旅舍，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某3。2019年5月31日15时45分，被告人马威茂驾驶云Ｆ×××××号中型客车在石屏县龙某公路1KM+100M处被民警查获，当场从其身上查获一包用透明塑料自封袋封装的毒品可疑物17粒、白色OPPO牌手机一部、人民币1000元。经称量和鉴定，查获的17粒甲基苯丙胺片剂可疑物净重1.59克，含甲基苯丙胺成分。经侦查实验，被告人马威茂贩卖的甲基苯丙胺片剂每粒净重0.09克。上述款物被石屏县公安局扣押，并随案移送白色OPPO牌手机一部、人民币1000元至本院。另查明：一、经尿液检测，被告人马威茂的尿液检测结果冰毒、吗啡均呈阳性。二、被告人马威茂在贩卖毒品的过程中，使用VIVO牌手机与买方联系。三、被告人马威茂到案后如实供述自己的犯罪事实，当庭自愿认罪。上述事实，经当庭举证、质证，有下列证据：一、石屏县公安局出具的抓获经过、现场检查笔录、物证照片、扣押决定书、扣押清单、受案登记表、立案决定书；石屏县人民检察院随案移送物品清单。证实：2019年5月31日15时45分，石屏县公安局民警在石屏县龙某公路1KM+100M处查获马威茂，当场从其右裤包内查获一包用透明塑料自封袋封装的毒品可疑物17粒、左后裤包内查获一张农行卡、右后裤包查获人民币1000元，从其驾驶的云Ｆ×××××号客车驾驶室旁垫子上查获一部白色OPPO牌手机。石屏县公安局扣押了查获的上述款物，并于同日立为刑事案件侦查。石屏县人民检察院随案移送白色OPPO牌手机一部、人民币1000元至本院。二、证人证言（一）证人李某的证言，主要内容为：2019年5月20日左右，他向“小马”买过五、六次“小麻”，有一次在石屏县文化站门口“小马”的班车上以160元（微信转账）买了4粒“小麻”；有一次在石屏县文化站门口“小马”的班车上以200元（150元微信转账、50元现金）买了6粒“小麻”；有一次在石屏县旧沙桥头以200元（微信转账）买了6粒“小麻”；其他的都是以100元3粒的价格，在石屏县文化站门口“小马”的班车上买。2019年5月28日13时30分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”。2019年5月29日13时40分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元价买了3粒“小麻”。2019年5月30日14时50分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”。（二）证人管某的证言，主要内容为：他吸食的“小麻”是在石屏县文化站门口的班车上向“小马”买的。2019年4月20日左右，他到石屏县文化广场门口向“小马”买了3粒“小麻”；另外两次也是2019年4月份，每次都是3粒，都是100元买的，都是他到班车上坐在“小马”后边，他递钱给“小马”，“小马”递东西给他。2019年5月份下旬向“小马”买了七、八次，都是微信红包或转账支付，最多的一次转账450元，购买了15粒，其余的300元、200元、100元不等，基本上都是以100元购买3粒的价格购买。2019年5月27日13时40分许，他到石屏县文化广场门口“小马”的班车上向“小马”以100元买了3粒“小麻”。2019年5月29日14时许，他到石屏县文化广场门口“小马”的班车上向“小马”以300元买了10粒“小麻”。（三）证人董某1的证言，主要内容为：2019年5月19日13时30分许，他在石屏县文化站门口“小马”的班车上向“小马”以120元（40元现金、80元微信转账）买了4粒“小麻”。2019年5月22日13时30分许，他在石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”；当天，他又在石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”；同日23时50分，他在峨山广场雕像处向“小马”以600元（微信转账）买了18粒“小麻”。（四）证人董某2的证言，主要内容为：2019年5月28日14时许，他在石屏县文化站门口“小马”的班车上向“小马”以200元买了5粒“小麻”。2019年5月29日14时许，他在石屏县文化站门口“小马”的班车上向“小马”以50元买了2粒“小麻”。自从他认识“小马”就开始向“小马”购买“小麻”，认识三年了，至少买过100粒左右“小麻”。（五）证人解某1的证言，主要内容为：两年前，他在龙武去峨山的岔路口“小马”的班车上向“小马”以160元买了4粒“小麻”；之后买了十多次，最少三两个，最多十个，价格有时35元1个，有时100元3个。（六）证人解某2的证言，主要内容为：2019年5月27日13时30分，他到石屏县文化站门口“小马”的班车上向“小马”以100元（微信转账）买了3粒“小麻”。2019年5月28日14时许，他到石屏县文化站门口“小马”的班车上向“小马”以100元（微信转账）买了3粒“小麻”。（七）证人董某3的证言，主要内容为：2019年5月29日14时许，他在石屏县仁和旅舍大门头上那个房间向“小马”以100元买了3粒“小麻”。三、被告人马威茂的供述和辩解，主要内容为：（一）他多次卖过“小麻”给李某。2019年4月份，有一次在石屏县文化广场门口其班车上以160元卖了4粒“小麻”给李某，有一次在石屏县文化广场门口其班车上以200元（50元现金、150元微信支付）卖了6粒“小麻”给李某，有一次在龙武旧沙××公路上以200元（微信支付）卖了6粒“小麻”给李某，后来在石屏县文化广场门口其班车上还卖过两次给李某，以每次100元3粒的价格卖了6粒。2019年5月28日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。2019年5月29日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。2019年5月30日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。（二）2019年4月份，他卖过三次“小麻”给管某，以每次100元3粒的价格卖了9粒“小麻”给管某。2019年5月下旬，在石屏县文化广场门口其班车上，以450元（微信支付）卖了15粒“小麻”给管某。2019年5月29日14时许，在石屏县文化广场门口其班车上以300元卖了10粒“小麻”给管某。2019年5月27日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给管某。（三）2019年5月19日13时30分许，他在石屏县文化广场门口其班车上以120元（40元现金、80元微信支付）卖了4粒“小麻”给董某1。2019年5月22日13时许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给董某1。2019年5月22日13时30分许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”。2019年5月22日23时许，他在峨山广场雕像处以600元（微信支付）卖了18粒“小麻”给董某1。（四）2019年5月28日14时许，他在石屏县文化广场门口其班车上以200元卖了5粒“小麻”给董某2。2019年5月29日14时许，他在石屏县文化广场门口其班车上以50元卖了2粒“小麻”给董某2。（五）一年前，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某1。前不久，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某1。（六）2019年5月28日13时许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某2。（七）2019年5月29日14时许，他在龙武某和旅舍以100元卖了3粒“小麻”给董某3。（八）他以前卖的“小麻”和被抓获时他装着的“小麻”是一模一样。四、辨认笔录及辨认照片，证实：管某、李某、解某1、解某2、董某3、董某1、董某2分别辨认出马威茂是卖“小麻”给其的人。马威茂分别辨认出李某、解某1、董某3、董某1、解某2是向其买“小麻”的人。五、毒品可疑物提取笔录、称量笔录及称量照片、侦查实验笔录及照片、鉴定委托书、毒品检验鉴定意见书、鉴定意见通知书等书证，证实：经称量，从马威茂处查获的毒品可疑物片剂17粒，净重1.59克；随机提取其中的1粒进行称量，净重0.09克。随机提取从马威茂处查获的毒品可疑物片剂作为检材送检，从检材中均检出甲基苯丙胺成分，鉴定意见已通知马威茂。六、尿液检验笔录及现场检测报告书，证实：2019年5月31日，民警依法对马威茂当场排出的尿液进行检测，检测结果冰毒、吗啡均呈阳性。七、户口证明、无违法犯罪记录证明，证实：马威茂的出生时间等基本自然情况，其作案时已达完全刑事责任年龄，在辖区居住期间无违法犯罪记录。以上证据来源合法，购买毒品人员的证言与被告人马威茂的供述中关于毒品交易时间、交易次数、交易数量存在部分不吻合，本院综合全案证据并根据证据之间相互能够印证、有利于被告人的原则作出认定，其它证据内容客观、真实，与案件事实相关联，具有证明力，本院予以确认。本院认为，被告人马威茂无视国家法律，违反国家对毒品管理的规定，向多人多次贩卖甲基苯丙胺共计11.94克，根据《中华人民共和国刑法》第三百五十七条“本法所称的毒品，是指鸦片、海洛因、甲基苯丙胺（冰毒）、吗啡、大麻、可卡因以及国家规定管制的其他能够使人形成瘾癖的麻醉药品和精神药品”、第三百四十七条第一款“走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚”、第三款“走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年有期徒刑以上有期徒刑，并处罚金”、第七款“对于多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算”的规定，被告人马威茂的行为已构成贩卖毒品罪，应据此规定予以相应的刑事处罚。公诉机关的指控事实清楚，证据确实充分，指控罪名成立。被告人马威茂被抓获后如实供述自己的罪行，根据《中华人民共和国刑法》第六十七条第三款的规定，系坦白，可以对其从轻处罚；其向多人多次贩卖毒品，可酌情从重处罚；其本人系吸毒人员，对从其处查获的毒品应当计入贩卖数量，但量刑时应考虑其吸食毒品的情节。在对被告人马威茂具体量刑时，本院根据马威茂贩卖毒品的事实、情节、社会危害程度、认罪悔罪态度等，予以综合考虑。公诉机关对量刑情节的认定准确、量刑建议适当，本院予以采纳。被告人马威茂“关于感觉没有贩卖过这么多毒品”的辩解意见，本院不予采信；其请求从轻判处的意见，本院予以综合考虑。辩护人张荣华关于“马威茂本人系吸毒人员，量刑时应考虑其吸食毒品的情节”以及“马威茂具有坦白情节”的辩护意见，本院予以采纳；其它辩护意见及量刑建议，与本院查明的事实及法律规定不符，本院不予采纳。公安机关扣押的毒品、毒资及犯罪工具，根据《中华人民共和国刑法》第六十四条的规定，应当予以没收。综上所述，本院为维护社会管理秩序，打击毒品犯罪，根据《中华人民共和国刑法》第三百五十七条、第三百四十七条第一款、第三款、第七款、第六十七条第三款、第六十四条之规定，判决如下：一、被告人马威茂犯贩卖毒品罪，判处有期徒刑九年，并处罚金人民币50000元。(刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2019年6月1日起至2028年5月31日止。罚金限判决生效之日一次性缴纳。)二、石屏县公安局扣押保管的17粒甲基苯丙胺片剂（净重1.59克），以及随案移送至本院的白色OPPO牌手机一部、人民币1000元，予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向云南省红河哈尼族彝族自治州中级人民法院提出上诉。书面上诉的，应交上诉状正本一份，副本二份。审　判　长　　王永才人民陪审员　　赵　昆人民陪审员　　杨官银二〇一九年十二月二十五日书　记　员　　白　龙 关注公众号“马 克 数 据 网”"

text_2 = "四川省崇州市人民法院 刑 事 判 决 书 （2019）川0184刑初625号 公诉机关四川省崇州市人民检察院。 被告人王磊，男，1984年3月6日出生，汉族，初中文化，住四川省崇州市。2019年4月26日因涉嫌犯非法持有毒品、容留他人吸毒罪被崇州市公安局刑事拘留，同年5月31日被逮捕。现羁押于崇州市看守所。 辩护人龙华，四川法邦律师事务所律师。 崇州市人民检察院以崇检公诉刑诉〔2019〕614号起诉书指控被告人王磊犯非法持有毒品罪、容留他人吸毒罪，于2019年12月23日向本院提起公诉。本院于同日立案，依法组成合议庭，适用简易程序，公开开庭审理了本案。崇州市人民检察院指派检察员杨萌出庭支持公诉，被告人王磊及其辩护人龙华到庭参加诉讼。现已审理终结。 崇州市人民检察院起诉指控：2019年4月19日左右，被告人王磊在崇州市容留吸毒违法人员赵某吸食毒品甲基苯丙胺（冰毒）；2019年4月22日左右，被告人王磊在唐兴客馆815房间容留吸毒人员赵某吸食冰毒；2019年4月25日，被告人王磊在崇州市容留吸毒违法人员岳某岚、塔某初吸食冰毒。 2019年4月25日11时许，崇州市公安局民警在对被告人王磊所租住的崇州市学苑东路229号1栋**房、**号出租房进行检查时，现场挡获吸毒人员王磊、岳某岚、塔某初，现场查获疑似毒品冰毒7袋、麻古3袋、吸毒工具一套。经称重，所查获的疑似毒品冰毒净重237.51克、麻古净重4.68克。后经成都市公安局物证鉴定所鉴定，上述毒品及吸毒工具内均含有甲基苯丙胺成分。 公诉机关认为，被告人王磊的行为应当以非法持有毒品罪、容留他人吸毒罪追究其刑事责任，建议对被告人王磊判处有期徒刑八年八个月至九年二个月，并处罚金三万四千元至五万四千元。公诉机关提交了公安机关的受案登记表和关于被告人王磊到案经过的说明，证人岳某岚、塔某初、赵某、倪某的证言，辨认笔录，鉴定意见，现场检查笔录和照片，扣押及保管清单，公安机关情况说明，被告人王磊的供述等证据证实，足以认定。 被告人王磊对指控的事实、罪名及量刑建议没有异议，同意适用简易程序且签字具结，在开庭审理过程中也无异议，自愿认罪认罚，但辩称自己曾检举孙某制造毒品的犯罪事实，应构成立功。辩护人龙华对指控的事实、罪名及量刑建议亦无异议，同意被告人王磊的辩解意见。 经审理查明的事实、证据与公诉机关的指控一致。另查明，公安机关出具了情况说明，证实公安机关已于2018年7月掌握孙某的犯罪线索并展开侦查，而被告人王磊是在2019年4月26日被羁押之后反映孙伟的犯罪线索。另，公安机关扣押在案被告人王磊随身携带的现金94,000元，在向银行缴纳时，被检查出其中一张百元面值为假币，已由银行没收，现扣押在案93,900元，暂存于崇州市公安局银行帐户；扣押手机两部（蓝色&ldquo;BIXING&rdquo;及黑色&ldquo;苹果&rdquo;手机各一部），以上物品均为被告人王磊个人财物；另扣押毒品甲基苯丙胺237.51克、麻古4.68克。扣押的手机与毒品均暂存于崇州市公安局涉案财物保管中心。 本院认为，被告人王磊违反国家毒品管理制度，非法持有含甲基苯丙胺成份的毒品，其行为构成非法持有毒品罪；同时被告人王磊为吸毒人员吸食毒品提供场所，其行为违反国家毒品管理制度，构成容留他人吸毒罪。崇州市检察院指控被告人王磊犯非法持有毒品罪和容留他人吸毒罪的事实和罪名成立，本院予以支持。关于被告人王磊提出自己检举孙某犯罪事实的行为是立功的辩解意见，本院认为，公安机关在被告人王磊检举孙某犯罪之前已经掌握了孙某的的犯罪线索，不属于立功，因此对被告人王磊的辩解意见不予采纳。辩护人龙华关于被告人王磊系初犯，归案后如实供述犯罪事实，自愿认罪认罚，请求对其从轻处罚的辩护意见与审理查明的事实一致，符合法律规定，予以采纳。被告人王磊一人犯数罪，予以数罪并罚。公诉机关量刑建议适当，予以采纳。根据被告人王磊犯罪的事实、性质、情节以及对于社会的危害程度，依照《中华人民共和国刑法》第三百四十八条、第三百五十四条、第六十九条、第六十七条第三款、第五十二条、第五十三条、第六十四条之规定，判决如下： 一、被告人王磊犯非法持有毒品罪，判处有期徒刑八年四个月，并处罚金三万六千元；犯容留他人吸毒罪，判处有期徒刑八个月，并处罚金四千元；数罪并罚，决定执行有期徒刑八年十个月，并处罚金四万元。 〔刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自二0一九年四月二十六日（羁押之日）起至二0二八年二月二十五日止。罚金限于本判决生效之次日起三日内缴纳。〕 二、对公安机关扣押在案的毒品甲基苯丙胺237.51克、麻古4.68克，由公安机关予以没收销毁；对公安机关扣押在案的手机两部（蓝色&ldquo;BIXING&rdquo;及黑色&ldquo;苹果&rdquo;手机各一部）发还被告人王磊，扣押在案的现金93,900元，用以执行罚金，剩余部分发还被告人王磊。 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接上诉于四川省成都市中级人民法院。书面上诉的，应当提交上诉状正本一份，副本二份。 审　判　长　　卫翔宇 人民陪审员　　吴　磊 人民陪审员　　钟　静 二〇一九年十二月二十四日 书　记　员　　龚　玲 附：《中华人民共和国刑法》 第三百四十八条非法持有鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的，处七年以上有期徒刑或者无期徒刑，并处罚金；非法持有鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。 第三百五十四条容留他人吸食、注射毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金。 第六十七条第三款犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。 第五十二条判处罚金，应当根据犯罪情节决定罚金数额。 第五十三条罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。 由于遭遇不能抗拒的灾祸等原因缴纳确实有困难的，经人民法院裁定，可以延期缴纳、酌情减少或者免除。 第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 第六十九条判决宣告以前一人犯数罪的，除判处死刑和无期徒刑的以外，应当在总和刑期以下、数刑中最高刑期以上，酌情决定执行的刑期，但是管制最高不能超过三年，拘役最高不能超过一年，有期徒刑总和刑期不满三十五年的，最高不能超过二十年，总和刑期在三十五年以上的，最高不能超过二十五年。 数罪中有判处附加刑的，附加刑仍须执行，其中附加刑种类相同的，合并执行，种类不同的，分别执行。 百度搜索“马 克 数 据 网”"

text_3 = "浙江省宁波市鄞州区人民法院 刑 事 判 决 书 （2019）浙0212刑初1388号 公诉机关浙江省宁波市鄞州区人民检察院。 被告人蒋磊，男，1983年9月19日出生于浙江省宁波市，汉族，初中文化，户籍所在地浙江省宁波市奉化区，住浙江省宁波市海曙区。2014年12月31日因吸毒被宁波市公安局海曙分局行政拘留五日。2015年3月19日因吸毒被宁波市公安局海曙分局行政拘留十五日。2015年6月17日因犯贩卖毒品罪被宁波市海曙区人民法院判处有期徒刑三年，并处罚金人民币一万元，2017年9月7日刑满释放。因涉嫌犯贩卖毒品罪于2019年3月20日被宁波市公安局鄞州分局传唤，次日被刑事拘留，同年4月26日被逮捕。现羁押于宁波市看守所。 辩护人王寅华，浙江甬港律师事务所律师，由宁波市鄞州区法律援助中心指派。 宁波市鄞州区人民检察院以甬鄞检刑诉〔2019〕1011号起诉书指控被告人蒋磊犯贩卖毒品罪，于2019年12月13日向本院提起公诉。本院于2019年12月17日立案，依法组成合议庭，适用普通程序，公开开庭审理了本案。庭审中被告人蒋磊表示自愿认罪，本案转为简易程序审理。宁波市鄞州区人民检察院指派检察员刘芙蓉出庭支持公诉，被告人蒋磊及其辩护人王寅华到庭参加诉讼。现已审理终结。 公诉机关指控： 2019年1月20日21时许，被告人蒋磊在宁波市海曙区荣安佳境小区门口，以人民币2000元的价格将2个甲基苯丙胺（冰毒）贩卖给陈某。 2019年3月4日18时许，被告人蒋磊在宁波市海曙区荣安佳境小区门口，以人民币1000元的价格将1个甲基苯丙胺贩卖给陈某。 同月20日，被告人蒋磊在宁波市海曙区被民警抓获。 上述事实，被告人蒋磊在开庭审理过程中亦无异议，且有证人陈某的证言，辨认笔录，技侦资料，刑事判决书，抓获经过，被告人蒋磊的身份证明等证据证实，足以认定。 本院认为，被告人蒋磊违反国家毒品管理法规，明知是毒品仍向他人非法贩卖，其行为已构成贩卖毒品罪。公诉机关指控的罪名成立。被告人蒋磊因贩卖毒品罪被判处有期徒刑，刑罚执行完毕后，五年内故意再犯应当判处有期徒刑以上的贩卖毒品罪，系累犯，又系毒品再犯，依法应当从重处罚。被告人蒋磊在庭审中能自愿认罪，可酌情从轻处罚。辩护人就此提出的辩护意见，本院予以采纳。根据被告人犯罪的事实、性质、情节和对于社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一款、笫四款、第三百五十六条、第六十五条第一款、第六十一条之规定，判决如下： 被告人蒋磊犯贩卖毒品罪，判处有期徒刑一年四个月，并处罚金人民币六千元。 （刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日。即自2019年3月20日起至2020年7月19日止。罚金限判决生效后一个月内向本院缴纳。） 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向浙江省宁波市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。 审　判　长　　俞露烟 人民陪审员　　王锡莉 人民陪审员　　吴丽君 二〇一九年十二月三十日 代书　记员　　郑　莹 关注公众号“马 克 数 据 网”"

text_4 = "贵州省仁怀市人民法院刑 事 判 决 书（2017）黔0382刑初357号公诉机关贵州省仁怀市人民检察院。被告人杨应贵,女，1943年4月17日出生于贵州省仁怀市，汉族，文盲，无业，住贵州省仁怀市。2017年4月15日被仁怀市公安局刑事拘留，同年5月23日因涉嫌犯贩卖毒品罪被执行逮捕。现羁押于遵义市第二看守所。仁怀市人民检察院以仁怀检公刑诉〔2017〕292号起诉书指控被告人杨应贵犯贩卖毒品罪，于2017年9月7日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。仁怀市人民检察院指派检察员刘元良、助理检察员余静出庭支持公诉，被告人杨应贵到庭参加了诉讼。现已审理终结。经审理查明，被告人杨应贵系吸毒人员。2017年4月14日15时许，被告人杨应贵在其位于仁怀市茅坝镇茅坝社区东风街住宅三楼厨房内，以200元的价格贩卖毒品海洛因0.01克给罗小虎后被公安机关查获。公安机关在被告人处查获毒品海洛因1.11克、电子秤一台、现金1800元和直板手机一部。上述事实，有受案登记表、立案决定书、户籍证明、扣押决定书及清单、抓获经过、情况说明、现场检测报告书、被告人的供述和辩解、证人证言、辨认笔录、检查笔录、毒品称量笔录、鉴定文书、毒品收据等证据在卷佐证，且被告人在开庭审理过程中亦无异议，足以认定。本院认为，被告人杨应贵违反国家对毒品的管制制度，明知是毒品海洛因而予以贩卖，其行为已触犯《中华人民共和国刑法》第三百四十七条的规定，构成贩卖毒品罪。公诉机关指控的事实和罪名成立，本院予以确认。被告人归案后能如实供述自己的罪行，庭审中认罪态度较好，有悔罪表现，可对其从轻处罚。公诉机关建议对被告人判处七个月以上一年六个月以下有期徒刑并处罚金的公诉意见与本院查明的量刑情节一致，本院予以采纳。据此，本院为保护公民的身心健康，维护正常的社会管理秩序，依照《中华人民共和国刑法》第三百四十七条第四款、第六十七条第三款、第六十四条以及《最高人民法院关于适用财产刑若干问题的规定》第二条第一款的规定，判决如下：一、被告人杨应贵犯贩卖毒品罪，判处有期徒刑七个月，并处罚金二千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日。即自2017年4月15日起至2017年11月14日止。）二、涉案工具手机一部、电子秤一台，予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向贵州省遵义市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审　判　长　　刘　科人民陪审员　　杨元龙人民陪审员　　程天亮二〇一七年九月二十一日书　记　员　　赵小方 更多数据：www.macrodatas.cn"

text_5 = "湖南省冷水江市人民法院刑 事 判 决 书（2016）湘1381刑初192号公诉机关湖南省冷水江市人民检察院。被告人周丽，绰号“丽姐”，女，1981年4月8日出生，汉族，大专文化，无业，户籍所在地冷水江市,现住冷水江市；曾因犯贩卖毒品罪，2015年3月12日，被本院判处有期徒刑七个月，并处罚金五千元，2015年7月22日释放；现因涉嫌犯贩卖毒品罪，2016年9月12日被冷水江市公安局刑事拘留，同年9月27日被执行逮捕；现押于娄底市看守所。冷水江市人民检察院以冷检公诉刑诉［2016］194号起诉书指控被告人周丽犯贩卖毒品罪，于2017年11月7日向本院提起公诉。本院受理后，依法组成合议庭在本院审判庭公开开庭进行了审理。书记员林亚辉担任庭审记录，冷水江市人民检察院指派检察员刘丹丹出庭支持公诉，被告人周丽到庭参加诉讼。本案现已审理终结。冷水江市人民检察院指控：1、2016年9月9日16时许,吸毒人员刘某1通过微信联系被告人周丽求购毒品。后刘某1发了200元微信红包给周丽，周丽驾车将毒品甲基苯丙胺和毒品甲基苯丙胺片剂用卫生纸包好，要“三毛”将毒品藏匿于冷水江市中心小学附近，并通知刘某1去取。后刘某1赶至中心小学，在学校对面的垃圾桶旁取到毒品。2、2016年9月11日17时许，周丽电话联系吸毒人员周某，问周某是否需要毒品，周某遂求购300元毒品，双方约定在冷水江市富贵家园见面。见面后，周某发了300元微信红包给周丽，周丽随即叫来一辆出租车，在车上周丽将一粒毒品甲基苯丙胺片剂和两小包毒品甲基苯丙胺给了周某。交易完成后，周某回到了家中。2016年9月11日22时许，公安根据群众举报，在冷水江市轧钢加油站附近的莫林风尚酒店对面抓获被告人周丽，从周丽裤子袋子里缴获1小包白色晶体可疑物和小部分红色片状可疑物。经鉴定，白色晶体可疑物、小部分红色片状可疑物检出甲基苯丙胺成分。2016年9月12日，民警到周某家，扣押了周丽于9月11日贩卖给周某的0.89克白色晶体可疑物和0.09克红色片状可疑物。经鉴定，白色晶体可疑物、小部分红色片状可疑物检出甲基苯丙胺成分。为证明上述事实，公诉机关提交了刑事判决书、释放证明、微信交易截图、通话清单、户籍资料、毒品收缴收据等书证，证人周某、刘某1的证言，被告人周丽的供述与辩解，提取笔录、检查笔录、称重取样笔录、辨认笔录，抓获经过，情况说明等证据。公诉机关认为，被告人周丽贩卖毒品甲基苯丙胺，其行为触犯了《中华人民共和国刑法》第三百四十七条第一款、第四款之规定，应当以贩卖毒品罪追究其刑事责任。被告人周丽曾因犯贩卖毒品罪被判处有期徒刑以上刑罚，刑罚执行完毕以后，在五年内再犯应该判处有期徒刑以上刑罚之罪，根据《中华人民共和国刑法》第六十五条第一款、第三百五十六条、第七十四条的规定，系累犯、毒品再犯，应当从重处罚，且不适用缓刑。到案后，周丽如实供述其罪行，依法可以从轻处罚。提请本院依法惩处。被告人周丽对起诉书指控的犯罪事实供认不讳。经审理查明：1、2016年9月9日16时许，吸毒人员刘某1通过微信向被告人周丽求购毒品，并以微信转账的方式向周丽支付了200元毒资。随后，周丽驾车在新化县桑梓镇附近地点从一名绰号为“二哥”的男子处购得了重约0.8克的毒品甲基苯丙胺（冰毒）及一粒毒品甲基苯丙胺片剂（俗称“麻古”）。周丽从上述购得的毒品中克扣了约三分之一的毒品甲基苯丙胺后，将剩余毒品交给了一名绰号叫“三毛”的男子。随后，“三毛”在周丽的要求下，将上述毒品藏匿在了冷水江市中心小学大门对面一垃圾桶内。后刘某1按照周丽的通知，取走了藏匿在该处的毒品。2、2016年9月11日17时许，周丽电话联系吸毒人员周某，询问其是否需要购买毒品，周某表示同意，并与周丽约定在冷水江市富贵家园附近地段交易。两人来到约定地点后，周某以微信转账的方式向周丽支付了300元毒资。随即，周丽带领周某搭乘一辆出租车，在出租车上，周丽将两小包毒品甲基苯丙胺和一粒毒品甲基苯丙胺片剂交给了周某。后周某将上述购得毒品带回至其居住的房屋，并未吸食。2016年9月11日22时许，冷水江市公安局民警在冷水江市轧钢加油站旁的莫林风尚酒店对面附近地段将被告人周丽抓获。民警当场从周丽身上缴获并扣押了1小包毒品甲基苯丙胺疑似物及2小包毒品甲基苯丙胺疑似物与毒品甲基苯丙胺片剂疑似物的混合物。次日，民警在冷水江市计生委对面周丰居住的房屋内缴获并扣押了前日周丽贩卖给周某的2小包毒品甲基苯丙胺疑似物及1粒毒品甲基苯丙胺片剂疑似物。经称重，从周丽处扣押的毒品甲基苯丙胺疑似物净重0.09克，扣押的毒品甲基苯丙胺疑似物与毒品甲基苯丙胺片剂疑似物的混合物净重1.26克；从周某处扣押的毒品甲基苯丙胺疑似物净重0.89克，扣押的毒品甲基苯丙胺片剂疑似物净重0.09克。经鉴定，上述毒品疑似物均检测出甲基苯丙胺成分。综上所述，被告人周丽共贩卖毒品2次，贩卖毒品甲基苯丙胺重约2.95克。上述事实，有由公诉机关提交并经庭审质证、认证的下列证据予以证明：1、户籍资料，证明被告人周丽的基本情况。2、刑事判决书、释放证明，证明周丽曾因犯贩卖毒品罪，2015年3月12日，被本院判处有期徒刑七个月，并处罚金五千元，2015年7月22日释放。3、抓获经过，证明被告人周丽系被抓获归案。4、微信截图，证明周某通过微信向被告人周丽求购毒品以及支付毒资的情况。5、通话清单，证明被告人周丽与吸毒人员周某、刘某3案发时间段内有电话联系。6、毒品收缴收据、检查笔录、提取笔录、称重取样笔录，证明民警从被告人周丽及吸毒人员周某处缴获并扣押了毒品甲基苯丙胺疑似物及毒品甲基苯丙胺片剂疑似物若干。7、毒品鉴定意见，证明民警从被告人周丽及吸毒人员周某处扣押的毒品疑似物检测出甲基苯丙胺成分。8、辨认笔录，证明周某、刘某1均辨认出了向其贩卖毒品的被告人周丽；周丽辨认出了向其购买毒品的周某、刘某1。9、证人刘某1的证言，证明2016年9月9日下午，他通过微信向周丽求购毒品并支付了200元毒资。当天晚上，他接到周丽的电话后，按照周丽的指示在冷水江市中心小学门口对面的一个垃圾桶内找到了藏着的毒品“冰毒”和“麻古”，“冰毒”重约0.6至0.7克，“麻古”一粒。10、证人周某的证言，证明2016年9月11日下午，周丽通过电话联系问他是否需要购买毒品，他表示同意。后来他按照周丽的要求来到富贵家园附近。他和周丽见面后，他通过微信向周丽支付了300元毒资。然后，周丽叫了一辆出租车。他和周丽上车后，周丽将两小包“冰毒”和一粒“麻古”交给了他。他拿到毒品后没有吸食，而是带回了家中。10、被告人周丽的供述亦证明上述事实，与其他证据相互印证。上述证据，符合证据合法性、客观性与关联性的要求，本院予以确认。本院认为,被告人周丽贩卖毒品甲基苯丙胺，其行为已构成贩卖毒品罪，应依法予以惩处。被告人周丽曾因犯贩卖毒品罪被判处有期徒刑，现又犯贩卖毒品罪，系毒品犯罪再犯，依法应当从重处罚。被告人周丽在有期徒刑刑罚执行完毕以后，五年以内再犯应当判处有期徒刑以上刑罚之罪，且前后两罪均系故意犯罪，犯前罪时已满十八周岁，系累犯，依法应当从重处罚。被告人周丽到案后如实供述其罪行，依法可以从轻处罚。综上所述，公诉机关指控的罪名成立，应当以贩卖毒品罪追究被告人周丽的刑事责任。本院依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第七款，第三百五十六条，第六十五条第一款，第六十七条第三款，第五十二条，第五十三条，《中华人民共和国刑事诉讼法》第一百九十五条第（一）项之规定，判决如下：被告人周丽犯贩卖毒品罪，判处有期徒刑一年十个月，并处罚金人民币八千元。（刑期自判决执行之日起计算。判决执行前先行羁押的，羁押一日折抵刑期一日，即自2016年9月12日起至2018年7月11日止；罚金限判决生效之日起三十日内缴纳。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖南省娄底市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本两份。审　判　长　　肖若谷人民陪审员　　梁荷清人民陪审员　　谢林文二〇一七年九月二十五日书　记　员　　林亚辉附件：本判决所适用的相关法律法规：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第三百五十六条因走私、贩卖、运输、制造、贩卖毒品罪被判过刑，又犯本节规定之罪的，从重处罚。第六十五条被判处有期徒刑以上刑罚的犯罪分子，刑罚执行完毕或者赦免以后，在五年以内再犯应当判处有期徒刑以上刑罚之罪的，是累犯，应当从重处罚，但是过失犯罪和不满十八周岁的人犯罪的除外。前款规定的期限，对于被假释的犯罪分子，从假释期满之日起计算。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第五十二条判处罚金，应当根据犯罪情节决定罚金数额。第五十三条罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。由于遭遇不能抗拒的灾祸等原因缴纳确实有困难的，经人民法院裁定，可以延期缴纳、酌情减少或者免除。《中华人民共和国刑事诉讼法》第一百九十五条在被告人最后陈述后，审判长宣布休庭，合议庭进行评议，根据已经查明的事实、证据和有关的法律规定，分别作出以下判决：（一）案件事实清楚，证据确实、充分，依据法律认定被告人有罪的，应当作出有罪判决；（二）依据法律认定被告人无罪的，应当作出无罪判决；（三）证据不足，不能认定被告人有罪的，应当作出证据不足、指控的犯罪不能成立的无罪判决。 更多数据：www.macrodatas.cn"

text_6 = "上海市虹口区人民法院刑 事 判 决 书（2017）沪0109刑初778号公诉机关上海市虹口区人民检察院。被告人朱某某，男，1983年11月30日出生，汉族，户籍所在地安徽省淮南市。上海市虹口区人民检察院以沪虹检诉刑诉〔2017〕710号起诉书指控被告人朱某某犯贩卖毒品罪，于2017年8月31日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。上海市虹口区人民检察院指派检察员周某出庭支���公诉，被告人朱某某到庭参加诉讼。现已审理终结。上海市虹口区人民检察院指控：被告人朱某某受“小天”(另案处理)指使，于2017年6月20日14时30分许，在本市虹口区西江湾路XXX号“虹口龙之梦”商场门口，将2小包白色晶体以人民币800元的价格贩卖给俞某，双方成交后被民警当场抓获。案发后，经上海市公安局虹口分局称重，上述2包白色晶体净重1.56克；经上海市毒品检验中心鉴定，上述2包白色晶体中均检出甲基苯丙胺成分。上述事实，被告人朱某某在开庭审理过程中无异议，并有证人俞某、辛某的证言，上海市公安局虹口分局出具的《扣押笔录》、《扣押清单》、《称量笔录》、《案发经过》、《工作情况》，拍摄的相关照片、视频及上海市毒品检验中心出具的《检验报告》等证据证实，足以认定。本院认��，被告人朱某某明知是毒品而予以出售，其行为已构成贩卖毒品罪。上海市虹口区人民检察院指控被告人朱某某犯贩卖毒品罪罪名成立。本案系共同犯罪。被告人朱某某到案后能如实供述自己的罪行，依法可从轻处罚。为维护国家对毒品的管制，保护公民身心健康不受侵犯，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第二十五条第一款、第六十七条第三款及第六十四条之规定，判决如下：一、被告人朱某某犯贩卖毒品罪，判处有期徒刑六个月，并处罚金人民币二千元。(刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年7月1日起至2017年12月30日止。罚金应自本判决生效之日起一个月内向本院缴纳。)二、缴获的毒品及犯罪工具一并予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向上海市第二中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判员　　马翠华二〇一七年九月十一日书记员　　倪帼英附：相关法律条文《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。……走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。……第二十五条共同犯罪是指二人以上共同故意犯罪。……第六十七条……犯罪嫌疑人��不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 来源：百度“马 克 数据网”"

text_7 = "湖北省红安县人民法院刑 事 判 决 书（2017）鄂1122刑初123号公诉机关湖北省红安县人民检察院。被告人柯桂菊，女，1969年1月4日出生，汉族，初中文化，无职业。出生地湖北省红安县，户籍所在地湖北省红安县，住湖北省红安县。因犯贩卖毒品罪，于2013年1月9日被武汉市江岸区人民法院判处有期徒刑六个月，并处罚金2000元，2013年3月14日刑满释放；因贩卖毒品罪，于2013年9月26日被武汉市江岸区人民法院判处有期徒刑八个月，并处罚金人民币4000元，2014年1月14日刑满释放。2017年3月30日因涉嫌贩卖毒品罪被刑事拘留，2017年5月5日被执行逮捕。现押黄冈市看守所。湖北省红安县人民检察院以鄂红检刑诉[2017]97号起诉书指控被告人柯桂菊犯贩卖毒品罪，于2017年8月2日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。湖北省红安县人民检察院指派检察员夏利和出庭支持公诉，被告人柯桂菊到庭参加了诉讼。本案现已审理终结。公诉机关指控：2016年4月至2017年3月期间，被告人柯桂菊在位于红安县八里湾镇阳福社区8号楼1单元4-1室的住处先后向红安县居民何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某等人销售麻果和冰毒等毒品牟利。2017年3月29日，红安县公安局干警现场抓获柯桂菊，并从其随身携带的黑色提包中查获59颗（共重5.51克）和一袋冰毒毒品（重0.27克）。随后，红安县公安局干警在被告人柯桂菊位于红安县红安县八里湾镇阳福社区8号楼1单元4-1室的住处搜查查获一颗麻果毒品（重0.11克）。经鉴定：从被告人柯桂菊随身携带的黑色提包中和住处查获、扣押的麻果和冰毒均检验出甲基苯丙胺毒品成分。公诉机关认为，被告人柯桂菊的行为构成贩卖毒品罪。被告人柯桂菊辩称，对公诉机关指控其犯贩卖毒品罪没有异议，但只向两人贩卖过毒品，一个是赵某，一个不知道叫什么名字，外号“黑子”或“黑皮”。不认识何某、韩某、刘某、徐某、胡某、袁某、张某、周某等人，也没有卖过毒品给他们。经审理查明：2016年4月至2017年3月期间，被告人柯桂菊在位于红安县八里湾镇阳福社区8号楼1单元4-1室的住处先后向红安县居民何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某等人销售麻果和冰毒等毒品牟利。2017年3月29日，红安县公安局干警现场抓获柯桂菊，并从其随身携带的黑色提包中查获59颗（共重5.51克）和一袋冰毒毒品（重0.27克）。随后，红安县公安局干警在被告人柯桂菊位于红安县红安县八里湾镇阳福社区8号楼1单元4-1室的住处搜查查获一颗麻果毒品（重0.11克）。经鉴定：从被告人柯桂菊随身携带的黑色提包中和住处查获、扣押的麻果和冰毒均检验出甲基苯丙胺毒品成分。上述事实，有经过法庭举证、质证的以下证据证实：1、证人何某的证言：我们都叫她嫂子或二嫂，红安口音，大约四十多岁，齐肩短发，一般身材。手机号码182××××9796。我在嫂子手中买过毒品。一次是2017年二月底，别人告诉问我的，把我带到红安县经济开发区二号路往八里方向去的一个小区里面，说四楼住着一个嫂子手中有毒品卖，然后我一个人上去，敲门，里面有个女的，问我做什么，我说买东西（毒品），她就问我要多少，我说两百元的，然后将200元钱给她，过了一会儿，她就从靠楼梯间的厕所窗户递出一包东西给我。里面装着四颗麻果。第二次是今年三月初的时候，我自己开车去的，以同样的方式买了一百元的毒品麻果两颗，并要到了嫂子的电话号码。第三次我先跟嫂子发信息，问她是否在家。嫂子回短信在家，我就开车过去，以上述相同的方式买了150元的毒品，麻果三颗。2017年3月24、5号的时候，韩某跟我打电话，要我帮忙带点毒品回去。我就直接去嫂子家，说要三百元钱的毒品，并将三百元给嫂子，她递给我四颗麻果和一袋子“冰”。我回到上新集，给了韩某三颗麻果和一袋子冰，自己留了一颗，韩某的三百元钱现在还没有给我。2、证人韩某的证言：3月25、6号左右，我叫何某跟我带了四颗麻果和一袋冰，他自己留了一颗，听她说是在二嫂手中买的。我认识二嫂，是个女的，住在觅儿工业园的一个小区里面，40多岁。我在二嫂手中拿过毒品。第一次是2016年4月份的时候，是陈佳（音）带我上去的，陈佳敲门，我跟陈佳将钱从窗户递进去，二嫂从窗户将毒品递出来。最近的一次是3月十几号的时候，我和何某一起去的，何某上楼拿货，我在下面等着。3、证人赵某的证言：我在八里工业园区阳福社区开了一家湖南米粉的早餐店。我在二嫂的手中购买过毒品，二嫂叫什么我不清楚，有40多岁，身高160左右，中等身材，卷发，染成黄色，长短刚到肩膀，红安本地口音。我一般就直接去二嫂居住的阳福社区四楼她家里去敲门或者打电话，她每次都是通过他家大门旁边的小窗户跟我沟通，我告诉她钱的金额后，她就将毒品给我，我就将现金通过窗户给她。4、证人刘某的证言：我与二嫂是在武汉做事的时候认识的，有二十年了，她大概四十多岁，真名字叫什么我不清楚，是红安人，住在工业园区阳福社区一栋居民楼的四楼。我在二嫂手中买过毒品。第一次是2017年春节前后，一天下午，我到二嫂居住的楼下，给她打电话说买点货，她问我要白的还是红的，我说都要，然后她从楼上给我送下来，一颗麻果一袋冰，我给了她100元。第二次是在第一次过后十几天，也是下午，我给二嫂打电话要买点麻果和冰，然后来到她家楼下，她从四楼将包好的毒品丢下来，我将100元钱放在她家楼下草坪的草下面就走了。第三次是2017年3月15日左右，我到了二嫂楼下，给她打电话说买多斤，她问我在什么地方，说她在外面买东西，然后在她们社区第一排房子旁边的过道上，卖给我一颗麻果和一袋冰，我给了她一百元钱。5、证人徐某的证言：二嫂大名叫可柯桂菊，是红安人，大概四十多岁，住在阳福社区第二排房子的四楼，电话是182××××9796，认识的人都叫她二嫂。我和她比较熟，是以前在武汉的时候认识的。我在柯桂菊手中买过两次毒品，第一次是2016年年底的时候，有一天下午我一个朋友从广州回来，他吸食毒品，我就给柯桂菊打电话说买点货。然后我来到她住的四楼，隔着窗户，她递给我一个塑料带，里面包着两颗麻果，我递给她100元现金。第二次是2017年春节后，我给柯桂菊打电话说要买点货，然后来到她住的四楼，还是跟上次一样，她隔着窗户递出一个包着毒品的塑料袋，里面包着两颗麻果，我从窗户递给她100元钱。6、证人胡某的证言：二嫂真名叫什么我不知道，是个四十多岁的女性，住在工业园区阳福社区8栋一单元401室，我有她的微信，在她被抓以后，我就将她的微信删了。我在二嫂手中买过毒品。第一次是2016年4月份的一天晚上，我和“猫子”开车来到二嫂楼下，我给了“猫子”300元钱上楼去买的一袋冰和四颗麻果。7、证人袁某（猫子）的证言：柯桂菊，女，一般大家都叫她二嫂，大概五十岁左右，住红安县工业园区阳福社区一栋楼的四楼。我跟她认识一年多了，是一个朋友“眼镜”带去买毒品时认识的，那是2016年的一天下午，我想要毒品，我听说眼镜在二嫂那里可以买到货，就给眼镜打电话，叫他带我去买。眼镜骑摩托车带我到阳福社区二嫂家楼下，上到四楼，眼镜敲门，隔着门跟二嫂说要货，买了两颗麻果、一袋冰，钱是我从窗户递进去的，二嫂将毒品包好从窗户递出来。8、证人张某的证言：二嫂大名好像叫柯桂菊，四十多岁，住工业园区阳福社区一栋楼的四楼，电话182××××9796，微信号叫“女王攻”。杜某是柯桂菊的儿子，大概二十多岁，跟二嫂住一栋楼，五楼。我在杜某手中买过三、四次毒品，他的毒品是从他母亲二嫂那里拿的。2017年3月份以前，买毒品的人都找柯桂菊，三月份以后，杜某开始卖毒品，但是不是帮他妈卖我就不清楚了。我是2015年经“猫子”介绍到柯桂菊手上买毒品。第一次是2015年，当时柯桂菊还住在觅儿岔路的旅馆那里，一天晚上，猫子带我到柯桂菊的住处，我在外面站着，给了180元给猫子，猫子进去找柯桂菊，过了一会儿，柯桂菊将三颗麻果包好送出来。9、证人周某的证言：我是通过“猫子”介绍认识二嫂的，二嫂大概四十多岁，女性，住工业园5号路旁边一个小区的四楼，到了那里我认得出来，她的电话和微信我都删除了。我在二嫂手中买多两次毒品，一次是2016年12月份的时候，一天下午，我先给二嫂打电话，说要四颗红的（指麻果），然后通过微信给二嫂转了200元钱，我来到二嫂居住的楼下，她从四楼阳台将用纸包好的麻果从楼上丢下来。第二次是在第一次之后2、3天，跟上次一样，我先打电话说要两颗红的，然后微信转100元给她，她从四楼将用纸包好的毒品丢下来。10、辨认笔录，证人何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某均在见证人的见证下，从12张年龄相仿的女性免冠照片中辨认出柯桂菊就是向他们贩卖毒品的“二嫂”。以及柯桂菊辨认出赵某是在她手中买过毒品的人。11、搜查笔录，证实对柯桂菊家搜查，并搜查出疑似毒品红色颗粒物一颗。12、扣押物品清单、扣押决定书，证实2017年3月29日，公安机关被告人柯桂菊持有的疑似麻果60颗（红色58颗、绿色2颗）、疑似冰毒一袋（白色晶体）。13、、称重笔录，在见证人和被告人柯桂菊参与下，公安机关对扣押的疑似毒品进行称重，麻果5.62克、冰毒0.27克，共计5.89克。取样笔录，证实在见证人和被告人柯桂菊参与下，公安机关对疑似毒品进行取样、密封、盖章，并准备送鉴定中心鉴定。14、黄冈市公安司法鉴定中心检验报告，对公安机关送检的检材鉴定均含有甲基苯丙胺毒品成分。15、到案经过证实被告人柯桂菊系被抓获归案。16、户籍证明证实被告人柯桂菊的个人户籍信息。17、武汉市江岸区人民法院刑事判决书两份，证实被告人柯桂菊曾因犯贩卖毒品罪被判处刑罚情况。18、红安县公安局办案说明，证实在柯桂菊随身携带提包中搜出疑似麻果59颗、疑似冰毒一袋。并证实，在对柯桂菊住处搜查过程中，赵某敲门找柯桂菊，称是想买毒品的事实。19、被告人柯桂菊的供述和辩解：（2017年3月29日供述）毒品卖给过一个男子三次，在红安工业园区阳福社区开早餐店，店名“湖南米粉”，就在我家附近。第一次应当是2017年3月初的时候，他到我家门口，敲门说要买麻果，我问他怎么知道我有，他说是别人告诉他的。我问他要多少毒品，他说150元钱的，我就卖给他三颗麻果。第二次在大约过了一个礼拜左右，他又来敲我家门，问我有没有东西（毒品），我问他要多少，他说两颗红的（麻果），一点白的（冰），我就卖给他了。第三次是3月26、7号左右，他又来敲门，买了四颗麻果。我们都是现金交易，我通过我家侧面的窗户将毒品递给他。还有一个叫黑皮的人，微信号×××，我不知道叫什么，3月27日晚上通过微信转账，买了250元钱，五颗麻果。我也是通过我家窗户递给他的。还有些人我不认识，也不知道叫什么，他们都只是来买一、两次，每次都是一百至两百元钱。（2017年4月18日供述）我是2017年春节以后开始贩卖毒品的，是为赚钱供我自己吸食毒品。我卖给过四个人，其中有一个是在八里开米粉店的外地男子，就具体叫什么我不清楚，他在我手中买过三次，前后大概买了500元左右的毒品；再就是一个外号叫黑子的人，在我这里买过几遍，大概400元左右的麻果；还有两个男子也在我这里买过麻果，具体细节记不清了。本院认为，被告人被告人柯桂菊无视国法，多次向多人贩卖毒品，情节严重，其行为构成贩卖毒品罪。公诉机关指控成立，本院予以支持。被告人柯桂菊被判处有期徒刑刑罚执行完毕后，五年内再犯应当判处有期徒刑以上之罪，是累犯，应当从重处罚。被告人柯桂菊称没有卖毒品给那么多人的辩解意见，因有证人证言的合理陈述以及辨认笔录等证据证实其多次向多人贩卖毒品的事实，对被告人的该辩解意见本院不予采信。据此，依照《中华人民共和国刑法》第三百四十七条第四款之规定，判决如下：被告人柯桂菊犯贩卖毒品罪，判处有期徒刑五年六个月，并处罚金人民币5000元。罚金于本判决生效之日起十日内缴纳.（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年3月30日起至2022年9月29日止。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖北省黄冈市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判长　　李平辉审判员　　曾志斌审判员　　刘松涛二〇一七年九月十八日书记员　　余爱军 马 克 数 据 网"

text_8 = "湖南省长沙市天心区人民法院刑 事 判 决 书(2017)湘0103刑初504号公诉机关长沙市天心区人民检察院。被告人曹俊朝,男,1994年11月15日出生于湖南省双峰县。2017年6月5日被抓获，因涉嫌贩卖毒品罪,次日被长沙市公安局天心分局刑事拘留;于2017年7月11日经长沙市天心区人民检察院批准,次日被长沙市公安局天心分局执行逮捕。现羁押于长沙市第一看守所。辩护人任孝，湖南天地人律师事务所律师。被告人杨嘉,男,1996年1月11日出生于长沙市。2017年6月5日被抓获，因涉嫌贩卖毒品罪,次日被长沙市公安局天心分局刑事拘留;于2017年7月11日经长沙市天心区人民检察院批准,于次日被长沙市公安局天心分局执行逮捕。现羁押于长沙市第一看守所。辩护人代勇贤，湖南天地人律师事务所律师。长沙市天心区人民检察院以长天检诉刑诉（2017）520号起诉书指控被告人曹俊朝、杨嘉犯贩卖毒品罪，于2017年9月12日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。长沙市天心区人民检察院指派检察员焦焰出庭支持公诉，被告人曹俊朝及其辩护人任孝、被告人杨嘉及其辩护人代勇贤均到庭参加诉讼。现已审理终结。长沙市天心区人民检察院指控：2017年6月,被告人曹俊朝从他人处购买甲基苯丙胺(冰毒)、甲基苯丙胺片剂(俗称“麻古”),并安排被告人杨嘉寻找购买毒品的人员。2017年6月4日晚,被告人杨嘉的朋友“大姚”(另案处理)电话联系被告人杨嘉要购买毒品,被告人杨嘉告知被告人曹俊朝,被告人曹俊朝与“大姚”约定毒品交易价格、地点后,用卫生纸包装少量甲基苯丙胺(冰毒)和1小粒甲基苯丙胺片剂(俗称“麻古”)交给被告人杨嘉,被告人杨嘉骑被告人曹俊朝的摩托车至本市天心区南托岭纵鑫网吧门口与“大姚”安排来交易毒品的蔡某见面,被告人杨嘉将上述毒品以260元的价格贩卖给蔡某,并将收取的毒资260元交给被告人曹俊朝,被告人曹俊朝分给被告人杨嘉60元作为报酬。因“大姚”要求还要购买1套毒品,被告人曹俊朝将少量甲基苯丙胺(冰毒)和1小粒甲基苯丙胺片剂(俗称“麻古”)交给被告人杨嘉。2017年6月5日下午15时许,被告人杨嘉与蔡某电话联系后,至南托岭纵鑫网吧2楼准备再次贩卖毒品给蔡某时被公安民警查获,公安民警起获被告人杨嘉的0.32克白色晶体、0.06克红色药丸。当日晚,被告人杨嘉协助公安民警抓获被告人曹俊朝,公安民警从被告人曹俊朝身上起获0.53克白色晶体、0.1克红色药丸。经司法鉴定,从被告人曹俊朝、杨嘉处起获的白色晶体均含有甲基苯丙胺成分,起获的红色药丸均检出甲基苯丙胺、咖啡因成分。二被告人归案后均如实供述自己的罪行。该院认为，被告人曹俊朝、杨嘉违反国家关于毒品管理的规定，明知甲基苯丙胺、甲基苯丙胺片剂系国家禁止贩卖的毒品而予以非法贩卖，其行为已触犯《中华人民共和国刑法》第三百四十七条第一、四款，应当以贩卖毒品罪追究其刑事责任。本案系共同犯罪,被告人曹俊朝系主犯,适用《中华人民共和国刑法》第二十五条第一款、第二十六条第四款;被告人杨嘉系从犯,适用《中华人民共和国刑法》第二十五条第一款、第二十七条,具有法定从轻处罚情节。被告人杨嘉协助公安机关抓获同案犯,具有立功情节,适用《中华人民共和国刑法》第六十八条。该院为上述指控提供了相关证据，并建议本院对被告人曹俊朝、杨嘉犯贩卖毒品罪分别在有期徒刑一年以下与有期徒刑八个月以下予以量刑，并处罚金。被告人曹俊朝、杨嘉对公诉机关指控的事实当庭供认属实；对公诉人当庭出示和宣读的证据，表示没有异议。被告人曹俊朝的辩护人提出以下辩护意见：1、被告人曹俊朝归案后能如实供述自己的罪行，具有坦白情节，依法可以减轻处罚。2、本案中公诉机关指控的第一笔涉案毒品并未查货，没有直接证据证明该笔毒品的重量，根据疑罪从无和有利于被告人原则，对该笔指控应该从轻处罚；3、公诉机关指控的2017年6月5日杨嘉贩毒的行为，被告人曹俊朝并不知情，与被告人曹俊朝无关；指控的6月6日的贩毒行为，被告人曹俊朝并没有真实交易，且交易行为尚未完成。4、被告人曹俊朝系初犯，认罪态度好，且家庭责任较重，恳请法庭从轻处罚。被告人杨嘉的辩护人提出以下辩护意见：1、被告人杨嘉自愿认罪，具有坦白情节，依法可以从轻处罚。2、被告人杨嘉协助公安机关抓获同案犯，具有立功情节。3、被告人杨嘉贩卖的毒品数量极少，对社会和个人造成很小的危害。4、被告人杨嘉年龄不大，属青年人犯罪，是初犯，偶犯，量刑时应从教育挽救的角度出发，从轻量刑。经审理查明，公诉机关指控被告人曹俊朝、杨嘉犯罪的事实属实，情节没有出入，本院予以确认。上述事实，被告人曹俊朝、杨嘉在开庭审理过程中亦无异议，并有查获的毒品照片、手机通话详单、抓获经过、扣押决定书、扣押物品清单、户籍资料、证人蔡某的证言、被告人曹俊朝、杨嘉的供述和辩解、鉴定意见、搜查笔录、称量笔录、辨认笔录、视听资料、被告人曹俊朝与“大姚”的微信聊天记录、毒品称重同步录音录像、讯问同步录像光盘等证据证实，足以认定。本院认为，被告人曹俊朝、杨嘉违反国家关于毒品的管理规定，明知甲基苯丙胺、甲基苯丙胺片剂系国家禁止贩卖的毒品而予以非法贩卖，其行为已构成贩卖毒品罪。长沙市天心区人民检察院指控被告人曹俊朝、杨嘉犯贩卖毒品罪的事实及罪名成立，本院应对二被告人的行为依法予以惩处。被告人曹俊朝的辩护人提出公诉机关指控的2017年6月5日杨嘉贩毒的行为，被告人曹俊朝并不知情，与被告人曹俊朝无关的辩护意见，经查，依据被告人曹俊朝、杨嘉的供述以及二被告人与“大姚”之间的手机通话详单、微信聊天记录等证据，均能证实被告人曹俊朝对6月5日的杨嘉与购毒者交易毒品行为是知情和默认的，故辩护人提出的该辩护意见本院不予采纳。被告人曹俊朝的辩护人提出被告人曹俊朝在控制下交易毒品行为，未造成社会危害的辩护意见，本院在量刑时酌情予以考虑。本案系共同犯罪，被告人曹俊朝系主犯；被告人杨嘉系从犯，应当从轻处罚。被告人杨嘉协助公安机关抓获同案犯，具有立功情节，依法可以从轻处罚。被告人杨嘉的辩护人提出的此辩护意见，本院予以采纳。被告人曹俊朝、杨嘉归案后能如实供述自己的罪行，依法可以从轻处罚。二辩护人提出的此辩护意见，本院予以采纳。二辩护人提出被告人系初犯，认罪态度好，家庭和个人情况特殊的辩护意见，本院在量刑时予以考虑。据此，依照《中华人民共和国刑法》第三百四十七条第一、四款、第二十五条第一款、第二十六条第一、四款、第二十七条、第六十八条、第六十七条第三款之规定，判决如下：一、被告人曹俊朝犯贩卖毒品罪，判处有期徒刑九个月，并处罚金人民币三千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年6月5日起至2018年3月4日止。罚金限在本判决生效后十日内缴清。）二、被告人杨嘉犯贩卖毒品罪，判处拘役四个月，并处罚金人民币一千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年6月5日起至2017年10月4日止。罚金限在本判决生效后十日内缴清。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖南省长沙市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本两份。审判员　　马铁夫二〇一七年九月十八日书记员　　钟　真附相关法律条文：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：(一)走私、贩卖、运输、制造鸦片一千克以上、氯胺酮或者甲基苯丙胺五十克以上或者其他毒品数量大的;(二)走私、贩卖、运输、制造毒品集团的首要分子;(三)武装掩护走私、贩卖、运输、制造毒品的;(四)以暴力抗拒检查、拘留、逮捕，情节严重的;(五)参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、氯胺酮或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、氯胺酮或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金;情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第二十五条共同犯罪是指二人以上共同故意犯罪。二人以上共同过失犯罪，不以共同犯罪论处；应当负刑事责任的，按照他们所犯的罪分别处罚。第二十六条组织、领导犯罪集团进行犯罪活动的或者在共同犯罪中起主要作用的，是主犯。三人以上为共同实施犯罪而组成的较为固定的犯罪组织，是犯罪集团。对组织、领导犯罪集团的首要分子，按照集团所犯的全部罪行处罚。对于第三款规定以外的主犯，应当按照其所参与的或者组织、指挥的全部犯罪处罚。第二十七条在共同犯罪中起次要或者辅助作用的，是从犯。对于从犯，应当从轻、减轻处罚或者免除处罚。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第六十八条犯罪分子有揭发他人犯罪行为，查证属实的，或者提供重要线索，从而得以侦破其他案件等立功表现的，可以从轻或者减轻处罚;有重大立功表现的，可以减轻或者免除处罚。 百度搜索“马 克 数 据 网”"

texts = [text_0, text_1, text_2, text_3, text_4, text_5, text_6, text_7, text_8]



for text in texts:
    print(trim_and_fetch_facts(text)[0])

# Read Csv file, conduct data enrichment, and save csv file (not updated yet, see test run code)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("ExtractFacts").getOrCreate()

# Base path where the data is saved
base_path = "/mnt/processed_data_criminal_case_analysis"

# Define the set of Causes of Action to filter and sample
causes_of_action = {"drug_related"}

# Initialize a dictionary to hold the counts
cases_count = {}

# Iterate over each cause of action to read the saved DataFrames, filter, and count the cases
for cause in causes_of_action:
    path = f"{base_path}/{cause}/*/*.csv"
    
    # Read the saved data
    df = spark.read.csv(path, header=True, inferSchema=True)
    
    # Filter to get rows where TrialProcedure is '一审'
    df_filtered = df.filter(col("TrialProcedure").contains("一审"))
    
    # Apply the UDF to the DataFrame and extract results into separate columns
    df_processed = df_filtered.withColumn("processed", trim_and_fetch_facts(df_filtered.FullText))

    df_final = df_processed.select(
        "*",  # Keep existing columns
        col("processed.response_text").alias("response_text"),
        col("processed.trimmedType").alias("trimmedType"),
        col("processed.text_around_trimmed_point").alias("text_around_trimmed_point")
    ).drop("processed")  # Drop the 'processed' struct column
    
    # Write the modified DataFrame back to a CSV, preserving the original structure
    output_path = f"{base_path}/{cause}_DrugTypeAmount_March_15"
    df_final.write.csv(output_path, mode="overwrite", header=True)

    
    # Count the number of cases for the current cause of action and update the dictionary
    count = df_filtered.count()
    cases_count[cause] = count




## Test Run: Move the Data Enrichment out of Spark

In [0]:
import os
import pandas as pd

def process_files(base_path, cause):
    global total_rows_count, failed_fetch_count, na_in_response_count
    # Define the output folder based on the base path and cause
    output_folder = f"{base_path}/{cause}_DrugTypeAmount_March_19"
    
    # Path pattern to match CSV files for the cause of action
    path_pattern = os.path.join(base_path, cause)
    
    # List to collect DataFrames (if needed)
    dfs = []

    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(path_pattern)

    for file in files:
        # Check if the item is a directory and iterate through its contents
        if file.isDir():
            sub_files = dbutils.fs.ls(file.path)
            for sub_file in sub_files:
                if sub_file.name.endswith(".csv"):
                    # Reading CSV file into DataFrame
                    file_path = sub_file.path.replace("dbfs:", "/dbfs")  # Convert to local file path if necessary
                    df = pd.read_csv(file_path, on_bad_lines='skip')

                    
                    # Filter rows where 'TrialProcedure' column contains '一审'
                    df_filtered = df[df['TrialProcedure'].str.contains('一审', na=False)]

                    # Apply 'trim_and_fetch_facts' function to 'FullText' column
                    enriched_data = df_filtered['FullText'].apply(trim_and_fetch_facts)

                    if df_filtered.empty:
                        print(f"No data after filtering for {sub_file.name}. Moving to the next file.")
                        continue

                    if not isinstance(enriched_data.iloc[0], tuple) or len(enriched_data.iloc[0]) != 3:
                        print(f"Data structure mismatch in file: {file_path}")
                        continue
                    
                    # Expanding the enriched_data into separate columns
                    df_expanded = pd.DataFrame(enriched_data.tolist(), columns=['ResponseText', 'TrimmedType', 'TextAroundTrimmedPoint'], index=df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_expanded], axis=1)

                    total_rows_count += len(df_filtered)
                    failed_fetch_count += len(df_filtered[df_filtered['ResponseText'] == 'Failed to fetch response from API'])
                    na_in_response_count += len(df_filtered[df_filtered['ResponseText'].str.contains("NA", na=False)])
                    
                    
                    # Construct the output path for the enriched CSV file
                    output_file_path = 'dbfs:'+ os.path.join(output_folder, f"{os.path.basename(sub_file.name)}")
                    
                    # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
                    df_filtered.to_csv(output_file_path.replace("dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis"
causes_of_action = ["drug_related"]

for cause in causes_of_action:
    total_rows_count, failed_fetch_count, na_in_response_count = 0, 0, 0
    process_files(base_path, cause)

    # Print the counts for process failures.
    print(f"For {cause}, Total rows processed: {total_rows_count}")
    print(f"For {cause}, Failed API fetch count: {failed_fetch_count}")
    print(f"For {cause}, NA in response count: {na_in_response_count}")



# test Claude API output

In [0]:
import os
import pandas as pd


def process_files(base_path, cause):
    global total_rows_count, failed_fetch_count, na_in_response_count
    # Define the output folder based on the base path and cause
    output_folder = f"/mnt/processed_data_criminal_case_analysis/drug_related_Claude_response_March_19"


    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(base_path)

    for file in files:
        # Check if the item is a directory and iterate through its contents
            for sub_file in files:
                if sub_file.name.endswith(".csv"):
                    # Reading CSV file into DataFrame
                    file_path = sub_file.path.replace("dbfs:", "/dbfs")  # Convert to local file path if necessary
                    df = pd.read_csv(file_path, on_bad_lines='skip')

                    
                    # Filter rows where 'TrialProcedure' column contains '一审'
                    df_filtered = df[df['TrialProcedure'].str.contains('一审', na=False)]

                    # Apply 'trim_and_fetch_facts' function to 'FullText' column
                    enriched_data = df_filtered['FullText'].apply(trim_and_fetch_facts)

                    if df_filtered.empty:
                        print(f"No data after filtering for {sub_file.name}. Moving to the next file.")
                        continue

                    if not isinstance(enriched_data.iloc[0], tuple) or len(enriched_data.iloc[0]) != 3:
                        print(f"Data structure mismatch in file: {file_path}")
                        continue
                    
                    # Expanding the enriched_data into separate columns
                    df_expanded = pd.DataFrame(enriched_data.tolist(), columns=['ResponseText', 'TrimmedType', 'TextAroundTrimmedPoint'], index=df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_expanded], axis=1)

                    total_rows_count += len(df_filtered)
                    failed_fetch_count += len(df_filtered[df_filtered['ResponseText'] == 'Failed to fetch response from API'])
                    na_in_response_count += len(df_filtered[df_filtered['ResponseText'].str.contains("NA", na=False)])
                    
                    
                    # Construct the output path for the enriched CSV file
                    output_file_path = 'dbfs:'+ os.path.join(output_folder, f"{os.path.basename(sub_file.name)}")
                    
                    # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
                    df_filtered.to_csv(output_file_path.replace("dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis/finetuning_training_data/drug_related_sample_training_data/"
causes_of_action = ["drug_related"]

for cause in causes_of_action:
    total_rows_count, failed_fetch_count, na_in_response_count = 0, 0, 0
    process_files(base_path, cause)

    # Print the counts for process failures.
    print(f"For {cause}, Total rows processed: {total_rows_count}")
    print(f"For {cause}, Failed API fetch count: {failed_fetch_count}")
    print(f"For {cause}, NA in response count: {na_in_response_count}")